In [5]:
import numpy as np     
import pandas as pd     
from scipy import stats
import seaborn as sns
import math

%matplotlib inline

In [6]:
x1_size = 200
x1 = stats.chi2(2).rvs(x1_size)

x1_mean = np.mean(x1)
x1_var = np.var(x1)
x1_std = math.sqrt(x1_var) 

print(f"X1: mean = {x1_mean:.3f}; var = {x1_var:.3f}; std = {x1_std:.3f}; size = {x1_size:.3f}")

X1: mean = 1.926; var = 4.096; std = 2.024; size = 200.000


In [7]:
x2_size = 200
x2 = stats.chi2(2).rvs(x1_size)

x1_mean = np.mean(x1)
x1_var = np.var(x1)
x1_std = math.sqrt(x1_var) 

print(f"X1: mean = {x1_mean:.3f}; var = {x1_var:.3f}; std = {x1_std:.3f}; size = {x1_size:.3f}")

X1: mean = 2.076; var = 5.087; std = 2.256; size = 200.000
